In [1]:
from src.indexing import build_index_from_url, build_chunks_from_tree
import time
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package punkt to ../../data/ntlk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to ../../data/ntlk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
tree, folder_path = await build_index_from_url("https://en.wikipedia.org/wiki/peru", 1)
chunks = build_chunks_from_tree(tree)

Created a chunk of size 607, which is longer than the specified 512
Created a chunk of size 660, which is longer than the specified 512


In [ ]:
import os
import numpy as np
from langchain_huggingface import HuggingFaceEmbeddings
import faiss

class EmbeddingManager:
    def __init__(self, model_name='all-MiniLM-L6-v2', model_dir='data/models', index_dir='data/faiss_index'):
        self.model_name = model_name
        self.model_dir = model_dir
        self.index_dir = index_dir
        self.model = None
        self.index = None

        os.makedirs(self.model_dir, exist_ok=True)
        os.makedirs(self.index_dir, exist_ok=True)

    def load_or_download_model(self):
        """Load model from local directory or download if not exists"""
        model_path = os.path.join(self.model_dir, self.model_name)
        
        if os.path.exists(model_path):
            print("Loading model from local directory...")
            self.model = HuggingFaceEmbeddings(cache_folder = model_path, model_name = self.model_name)
        else:
            print("Downloading model...")
            self.model = HuggingFaceEmbeddings(cache_folder = model_path, model_name = self.model_name)
            self.model.save(model_path)
        return self.model

    async def generate_embeddings(self, chunks):
        """Convert text chunks to embeddings"""
        if not self.model:
            self.load_or_download_model()
            
        print("Generating embeddings...")
        return await self.model.aembed_documents(chunks)

    def normalize_embeddings(self, embeddings):
        """Normalize embeddings to unit length"""
        norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
        return embeddings / norms

    def create_faiss_index(self, embeddings):
        """Create and save FAISS index"""
        # Convert embeddings to float32 numpy array
        embeddings = np.array(embeddings).astype('float32')
        
        # Normalize embeddings
        embeddings = self.normalize_embeddings(embeddings)
        
        # Create FAISS index
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dimension)
        self.index.add(embeddings)
        print(f"Created FAISS index with {self.index.ntotal} vectors")
        return self.index

    def save_faiss_index(self, index_name='my_index'):
        """Save FAISS index to disk"""
        if self.index is None:
            raise ValueError("Index not initialized. Create index first.")
            
        index_path = os.path.join(self.index_dir, f"{index_name}.index")
        faiss.write_index(self.index, index_path)
        print(f"Index saved to {index_path}")

    def load_faiss_index(self, index_name='my_index'):
        """Load FAISS index from disk"""
        index_path = os.path.join(self.index_dir, f"{index_name}.index")
        if not os.path.exists(index_path):
            raise FileNotFoundError(f"No index found at {index_path}")
            
        self.index = faiss.read_index(index_path)
        print(f"Loaded index with {self.index.ntotal} vectors")
        return self.index

    async def __call__(self, chunks=None, index_name):
        """Check for existing index, load if exists; else create from chunks and save."""
        index_path = os.path.join(self.index_dir, f"{index_name}.index")
        if os.path.exists(index_path):
            self.load_faiss_index(index_name)
        else:
            if chunks is None:
                raise ValueError("Chunks must be provided to create a new index.")
            
            text_chunks = [doc.page_content for doc in chunks]
            embeddings = await self.generate_embeddings(text_chunks)
            self.create_faiss_index(embeddings)
            self.save_faiss_index(index_name)
        return self.index

c:\Users\Ruan Felipe\Desktop\llm-chunking\.venv\Lib\site-packages\IPython\core\compilerop.py:86: RuntimeWarning: coroutine 'EmbeddingManager.generate_embeddings' was never awaited
  return compile(source, filename, symbol, self.flags | PyCF_ONLY_AST, 1)


In [17]:
folder_path / "index.jz"

WindowsPath('data/websites/en.wikipedia.org_wiki_peru/index.jz')

In [ ]:
em = EmbeddingManager(index_dir = folder_path  )
# Generate embeddings
index = await em(chunks)

Loading model from local directory...
Generating embeddings...
Created FAISS index with 337 vectors


RuntimeError: Error in __cdecl faiss::FileIOWriter::FileIOWriter(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:102: Error: 'f' failed: could not open data\websites\en.wikipedia.org_wiki_peru\data\websites\en.wikipedia.org_wiki_peru.index for writing: No such file or directory

<coroutine object EmbeddingManager.__call__ at 0x0000019ED93F6020>